# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [145]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
import math

# global variable
prev_clip_data = [0, 0, 0, 0]        # Slope & intercept from previous clip used as reference to filter outliers
bottom_y = 0                         # Variable to hold the bottom most starting point

# constants
SLOPE_TOLERANCE = 0.5                # Used to derieve allowed slope range based on on previous clip slope
INTERCEPT_TOLERANCE = 25             # Used to derieve allowed intecept range based on on previous clip intercept
TOP_Y_FACTOR = 0.615                 # Factor that decides the top edge of mask
BOTTOM_Y_FACTOR = 1.0                # Factor that decides the bottom edge of mask
MASK_BOTTOM_RIGHT_X_FACTOR = 0.975   # Factor that decides the bottom right position of mask
MASK_BOTTOM_LEFT_X_FACTOR = 0.125    # Factor that decides the bottom left position of mask
MASK_TOP_RIGHT_X_FACTOR = 0.575      # Factor that decides the top right position of mask
MASK_TOP_LEFT_X_FACTOR = 0.425       # Factor that decides the top left position of mask

## Read in an Image

In [2]:
#reading in an image
#image = mpimg.imread('test_images/Challenge6.jpg')

#printing out some stats and plotting
#print('This image is:', type(image), 'with dimensions:', image.shape)
#plt.imshow(image)  # if you wanted to show a single color channel image called 'gray',
#    for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [170]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    This section iterates through the line co-ordinates returned by Hough transform and
    calculates the average slope (m) and intercept (b) for left lane and right lane seperately.
    Positive slope is considered as right lane and negative slope is considered as left lane.
    
    The following filters are applied :
        1. Lines with slope^2 outside the range of 0.1 and 100 are filtered out as horizontal lines
        2. Lines with slope < 0 (left lane) but lying to the right of center are filtered
        3. Lines with slope > 0 (right lane) but lying to the left of center are filtered
        4. Lines with slope & intercept beyond the allowed variance are filtered.

    The mean of slope and intercept of valid left side lines is calculated as the left lane slope & intecept.
    The mean of slope and intercept of valid right side lines is calculated as the left lane slope & intecept.

    The left and right lanes are extrapolated by determining top and bottom x values using y = mx + b
    for y = top and bottom edge of mask
    
    The extrapolated lines are then drawn to produce the image.
    """

    global prev_clip_data                    # Array that holds the slope & intercept of previous clip
    global bottom_y                          # Variable that holds the bottom most y value

    prev_left_slope = prev_clip_data[0]      # Variable that holds the slope of left lane from previous clip
    prev_left_intercept = prev_clip_data[1]  # Variable that holds the intercept of left lane from previous clip
    prev_right_slope = prev_clip_data[2]     # Variable that holds the slope of right lane from previous clip
    prev_right_intercept = prev_clip_data[3] # Variable that holds the intercept of right lane from previous clip

    left_lane_slope_accumulator = 0.0        # Variable used to accumulate left lane slope to calculate mean
    left_lane_intercept_accumulator = 0.0    # Variable used to accumulate left lane intercept to calculate mean
    left_lane_line_counter = 0               # Variable used to count the number of valid left lanes
    left_lane_intercept_average = 0.0        # Variable used to hold the left lane intercept average
    right_lane_slope_accumulator = 0.0       # Variable used to accumulate right lane slope to calculate mean       
    right_lane_intercept_accumulator = 0.0   # Variable used to accumulate right lane intercept to calculate mean
    right_lane_line_counter = 0              # Variable used to count the number of valid right lanes
    right_lane_slope_average = 0.0           # Variable used to hold the right lane intercept average
    right_lane_intercept_average = 0.0       # Variable used to hold the right lane intercept average

    top_y = int(img.shape[0] * TOP_Y_FACTOR) # Calculates the top edge of mask to be used in extrapolation

    if prev_left_slope == 0:                 # Identifies the very first clip and uses logic without prev clip reference
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope, intercept = np.polyfit([x1, x2], [y1, y2], 1)   # Calculate slope & intercept
                if((slope*slope) > 0.1) & ((slope*slope) < 100):       # Filters out horizontal lines 
#                    cv2.line(img, (x1, y1), (x2, y2), color=[0, 255, 0], thickness=2)
                    if y1 > bottom_y:                                  # Reposition bottom edge if line is lower
                        bottom_y = y1
                    if y2 > bottom_y:                                  # Reposition bottom edge if line is lower
                        bottom_y = y2
                    if slope < 0:                                      # Identifies lane position as left or right
                        left_lane_slope_accumulator += slope
                        left_lane_intercept_accumulator += intercept
                        left_lane_line_counter += 1
                    else:
                        right_lane_slope_accumulator += slope
                        right_lane_intercept_accumulator += intercept
                        right_lane_line_counter += 1
                else:
#                    print ('ignoring line ', line, ' as outlier ', slope, intercept)
#                    cv2.line(img, (x1, y1), (x2, y2), color=[0, 0, 255], thickness=2)
                    pass
        
        """
        Below section calculates the mean of slope and intercept for left and right lanes seperately using 
        filtered out lines.  
        """
        if left_lane_line_counter != 0:                                    # handle division by zero scenario
            left_lane_slope_average = left_lane_slope_accumulator / left_lane_line_counter
            left_lane_intercept_average = left_lane_intercept_accumulator / left_lane_line_counter
            left_lane_top_x = (top_y - left_lane_intercept_average)/ left_lane_slope_average
            left_lane_bottom_x = (bottom_y - left_lane_intercept_average) / left_lane_slope_average
            cv2.line(img, (int(left_lane_bottom_x), bottom_y), (int(left_lane_top_x), top_y), color, thickness)

        if right_lane_line_counter != 0:                                    # handle division by zero scenario
            right_lane_slope_average = right_lane_slope_accumulator / right_lane_line_counter
            right_lane_intercept_average = right_lane_intercept_accumulator / right_lane_line_counter
            right_lane_top_x = (top_y - right_lane_intercept_average) / right_lane_slope_average
            right_lane_bottom_x = (bottom_y - right_lane_intercept_average) / right_lane_slope_average
            cv2.line(img, (int(right_lane_bottom_x), bottom_y), (int(right_lane_top_x), top_y), color, thickness)

        prev_clip_data = [left_lane_slope_average, left_lane_intercept_average, 
                             right_lane_slope_average, right_lane_intercept_average]
#        print('current clip data =', prev_clip_data)

    else:
        left_lane_slope_accumulator = prev_left_slope                 # Use slope and intercept from previous
        left_lane_intercept_accumulator = prev_left_intercept         # clip as one of the valid lines. This  
        left_lane_line_counter = 1                                    # will help increase the weight for valid
        right_lane_slope_accumulator = prev_right_slope               # lines in current clip and also act as 
        right_lane_intercept_accumulator = prev_right_intercept       # reference in case no line could be detected
        right_lane_line_counter = 1
        
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope, intercept = np.polyfit([x1, x2], [y1, y2], 1)  # calculate slope & intercept
                if((slope*slope) > 0.1) & ((slope*slope) < 100):      # filter out horizontal lines
                    if slope < 0:                                     # Line slope < 0 is considered left lane line
                        if x1 < (img.shape[1]/2):                     # Check if line is in left side of image
                            """
                            The below condition checks if a line is within the tolerance range using previous
                            clip slope and intercept as reference
                            """
                            if ((prev_left_slope-SLOPE_TOLERANCE) < slope < (prev_left_slope+SLOPE_TOLERANCE)) & \
                             ((prev_left_intercept-INTERCEPT_TOLERANCE) < intercept < \
                                (prev_left_intercept+INTERCEPT_TOLERANCE)):
                                left_lane_slope_accumulator += slope
                                left_lane_intercept_accumulator += intercept
                                left_lane_line_counter += 1
#                                cv2.line(img, (x1, y1), (x2, y2), color=[0, 255, 0], thickness=20)
#                               print('green data left =', slope, intercept)
                                if y1 > bottom_y:                    # Reposition bottom edge if line is lower
                                    bottom_y = y1
                                if y2 > bottom_y:                    # Reposition bottom edge if line is lower
                                    bottom_y = y2
                            else:
#                                print('ignoring line ', line, ' as outlier ', slope, intercept)
#                               cv2.line(img, (x1, y1), (x2, y2), color=[0, 0, 255], thickness=2)
                                pass
                        else:
#                           print('ignoring line ', line, ' as outlier ', slope, intercept)
#                            cv2.line(img, (x1, y1), (x2, y2), color=[240, 66, 236], thickness=2)
                            pass
                    else:
                        if x1 > (img.shape[1]/2):                   # Check if line is in right side of image
                            """
                            The below condition checks if a line is within the tolerance range using previous
                            clip slope and intercept as reference
                            """
                            if ((prev_right_slope-SLOPE_TOLERANCE) < slope < (prev_right_slope+SLOPE_TOLERANCE)) & \
                                ((prev_right_intercept-INTERCEPT_TOLERANCE) < \
                                    intercept < (prev_right_intercept+INTERCEPT_TOLERANCE)):
                                right_lane_slope_accumulator += slope
                                right_lane_intercept_accumulator += intercept
                                right_lane_line_counter += 1
#                                cv2.line(img, (x1, y1), (x2, y2), color=[0, 255, 0], thickness=20)
#                                print('green data right =', slope, intercept)
                                if y1 > bottom_y:                        # Reposition bottom edge if line is lower
                                    bottom_y = y1
                                if y2 > bottom_y:                        # Reposition bottom edge if line is lower
                                    bottom_y = y2
                            else:
#                               print('ignoring line ', line, ' as outlier ', slope, intercept)
#                                cv2.line(img, (x1, y1), (x2, y2), color=[0, 0, 255], thickness=2)
                                pass
                        else:
#                           print('ignoring line ', line, ' as outlier ', slope, intercept)
#                            cv2.line(img, (x1, y1), (x2, y2), color=[240, 66, 236], thickness=2)
                            pass
                else:
#                   print('ignoring line ', line, ' as outlier ', slope, intercept)
#                    cv2.line(img, (x1, y1), (x2, y2), color=[252, 255, 111], thickness=2)
                    pass

        """
        Below section calculates the mean of slope and intercept for left and right lanes seperately using 
        filtered out lines.  
        """
        left_lane_slope_average = left_lane_slope_accumulator / left_lane_line_counter
        left_lane_intercept_average = left_lane_intercept_accumulator / left_lane_line_counter
        left_lane_top_x = (top_y - left_lane_intercept_average) / left_lane_slope_average
        left_lane_bottom_x = (bottom_y - left_lane_intercept_average) / left_lane_slope_average
        cv2.line(img, (int(left_lane_bottom_x), bottom_y), (int(left_lane_top_x), top_y), color, thickness)

        right_lane_slope_average = right_lane_slope_accumulator / right_lane_line_counter
        right_lane_intercept_average = right_lane_intercept_accumulator / right_lane_line_counter
        right_lane_top_x = (top_y - right_lane_intercept_average) / right_lane_slope_average
        right_lane_bottom_x = (bottom_y - right_lane_intercept_average) / right_lane_slope_average
        cv2.line(img, (int(right_lane_bottom_x), bottom_y), (int(right_lane_top_x), top_y), color, thickness)

        """
        Set the previous clip data to the current clip data so that it can be used for next clip
        """
        prev_clip_data = [left_lane_slope_average, left_lane_intercept_average,  
                             right_lane_slope_average, right_lane_intercept_average]

#        print('current clip data =', prev_clip_data)
    
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    draw_lines(line_img, lines)
    return line_img


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    Blends the orginial image with the image containing identified lane lines
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
import os
os.listdir("test_images/")

['Challenge5.jpg',
 'Challenge6.jpg',
 'Challenge7.jpg',
 'Output2_Challenge5-old.jpg',
 'Output2_Challenge5-old1.jpg',
 'Output2_Challenge5.jpg',
 'Output2_Challenge6-old.jpg',
 'Output2_Challenge6-old1.jpg',
 'Output2_Challenge6.jpg',
 'Output2_Challenge7-old.jpg',
 'Output2_Challenge7-old1.jpg',
 'Output2_Challenge7.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the test_images directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [169]:
def pipeline(input_image):
    """
    Pipeline that will draw lane lines on the test_images and then save them to the test_images directory.
    """
    
    gaussian_kernel_size = 7  # kernel size used for gaussian blur
    canny_low_threshold = 40 # pixel with gradient below the low threshold are rejected
    canny_high_threshold = 100 # gradient used to detect strong edges
    
    
    mask_bottom_right_x = input_image.shape[1] * MASK_BOTTOM_RIGHT_X_FACTOR
    mask_bottom_left_x = input_image.shape[1] * MASK_BOTTOM_LEFT_X_FACTOR
    mask_top_right_x = input_image.shape[1] * MASK_TOP_RIGHT_X_FACTOR
    mask_top_left_x = input_image.shape[1] * MASK_TOP_LEFT_X_FACTOR
    mask_top_y = input_image.shape[0] * TOP_Y_FACTOR
    mask_bottom_y = input_image.shape[0] * BOTTOM_Y_FACTOR

    
    vertices = np.array([[(mask_top_left_x, mask_top_y), (mask_top_right_x, mask_top_y),  \
                          (mask_bottom_right_x, mask_bottom_y), (mask_bottom_left_x, mask_bottom_y)]],  \
                             dtype=np.int32) # vertices of the masked area
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 60     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 30    # maximum gap in pixels between connectable line segments
    
    grayscale_image = grayscale(input_image)
#    print ("gray image")
#    plt.imshow(grayscale_image, cmap='gray')

    gaussian_blurred_image = gaussian_blur(grayscale_image, gaussian_kernel_size)
#    print ("gaussian image")
#    plt.imshow(gaussian_blurred_image, cmap='gray')

    canny_image = canny(gaussian_blurred_image, canny_low_threshold, canny_high_threshold)
#    print ("canny image")
#    plt.imshow(canny_image, cmap='gray')
#    plt.imsave('canny.png', canny_image, format='png', cmap='gray')

    masked_image = region_of_interest(canny_image, vertices)
#    print ("masked image")
#    plt.imshow(masked_image, cmap='gray')
#    plt.show()

    hough_image = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap)
#    print ("hough image")
#    plt.imshow(hough_image, cmap='gray')
#    plt.show()

    weighted_image = weighted_img(hough_image, input_image)
#    print ("weighted image")
#    plt.imshow(weighted_image)
    
    return weighted_image

'''
NOTE : This section is used to test the pipeline against the set of test images provided
'''

images = os.listdir('test_images/')

for image in images:
    
    if image[:6] != 'Output':
        input_image = mpimg.imread('test_images/%s' % image)
        print('processing image ', image)
        prev_clip_data = [0, 0, 0, 0]
        bottom_y = 0
        output_image = pipeline(input_image)
        output_img_name = 'Output2_' + image
        mpimg.imsave('test_images/%s' % output_img_name, output_image)
    else:
        print('ignoring image ', image)

processing image  Challenge5.jpg
processing image  Challenge6.jpg
processing image  Challenge7.jpg
ignoring image  Output2_Challenge5.jpg
ignoring image  Output2_Challenge6.jpg
ignoring image  Output2_Challenge7.jpg


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # Calling pipeline that will return the final output (image where lines are drawn on lanes)
        
    result = pipeline(image)

    return result

Let's try the one with the solid white lane on the right first ...

In [174]:
prev_clip_data = [0, 0, 0, 0]
bottom_y = 0
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 221/222 [00:06<00:00, 34.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 7.37 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [175]:
prev_clip_data = [0, 0, 0, 0]
bottom_y = 0
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 681/682 [00:16<00:00, 52.75it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 17.5 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [171]:
prev_clip_data = [0, 0, 0, 0]
bottom_y = 0
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:13<00:00, 25.45it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 14.9 s


## HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))